In [35]:
!pip install boltons -q

In [68]:
import numpy as np
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from boltons.iterutils import windowed
from tqdm import tqdm, tqdm_notebook
MAIN_PATH = '/Users/luyiming/Downloads/中文歌词语料/song_list_chinese.json'

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [26]:
song_data = json.loads(open(MAIN_PATH).read())
song_data = list(map(dict, song_data))
print(song_data[1]['lyric'])
print(type(song_data[1]))
print('song num : ', len(song_data))

我想我的脚步已不能停
要去山顶看很美很美的风景
我只是遗憾
再也不能和你去聆听
那支会孤单很久的风铃
抱歉 我不能带你去远行
如果想我了 就看天边最亮的星星
我只是遗憾
我的温暖你再也不能感应
最后唱首天使的翅膀给你听
相信你还在这里
从不曾离去
就算烧成灰烬 也化成春泥
今生今世 归去来兮
我会找个天使替我去爱你

抱歉 我不能带你去远行
如果想我了 就看天边最亮的星星
我只是遗憾
我的温暖你再也不能感应
最后唱首天使的翅膀给你听
相信你还在这里
从不曾离去
就算烧成灰烬 也化成春泥
今生今世 归去来兮
我会找个天使替我去爱你

相信你还在这里
从不曾离去
就算烧成灰烬 也化成春泥
今生今世 归去来兮
我会找个天使替我去爱你
相信你还在这里
从不曾离去
就算烧成灰烬 也化成春泥
今生今世 归去来兮
我会找个天使替我去爱你
今生今世 归去来兮
我会找个天使替我去爱你
<class 'dict'>
song num :  6828


In [32]:
jay = '林夕'
jay_song = []
for song in song_data:
    try:
        if song and jay in song.get('lyric', ''):
            jay_song.append(song['lyric'])
            print(song['lyric'])
            print('-'*100)
    except:1
print('jay song num : ', len(jay_song))

作曲 : 周杰伦
 作词 : 林夕
我没有胆挂念 你没有心见面 试问我可以去边
只要我出现 只怕你不便 亦连累你丢脸
你是我的秘密 我是你的废物 缺席也不算损失
今晚你生日 祝我有今日 地球上快消失
眼泪还是留给天抚慰 你是前度 何必听我吠
再不走 有今生 无下世 你是否想我 起这个毒誓
宁愿失恋亦不想失礼 难道要对着你力歇声嘶
即使不抵 都要眼闭 我这种身世 有甚么资格 献世

我共你不够熟 眼泪也比较浊 也没气质对你哭
不介意孤独 比爱你舒服 别离就当祝福
眼泪还是留给天抚慰 你是前度 何必听我吠
再不走 有今生 无下世 你是否想我 起这个毒誓
宁愿失恋亦不想失礼 难道要对着你力歇声嘶
即使不抵 都要眼闭 我这种身世 有甚么资格 献世

即使不抵 都要眼闭
我自卑不怕 有自尊只怕 怕献世
----------------------------------------------------------------------------------------------------
作曲 : Christopher Chak
 作词 : 林夕
分手时内疚的你一转脸
为日后不想有什么牵连
当我工作 睡觉 祷告 娱乐 那么刻意过好每天
谁料你见松绑了又愿见面

谁当初想摆脱被围绕左右
过后谁人被遥控于世界尽头
勒到呼吸困难才知变扯线木偶
这根线其实说到底 谁拿捏在手

不聚不散 只等你给另一对手擒获
那时青丝 不会用上余生来量度
但我拖着躯壳 发现沿途寻找的快乐
仍系于你肩膊 或是其实在等我舍割
然后断线风筝会直飞天国

这些年望你紧抱他出现
还凭何担心再互相纠缠
给我找个伴侣找到留下你的足印也可发展
全为你背影逼我步步向前

如一根丝牵引着拾荒之路
结在喉咙内痕痒得似有还无
为你安心我在微笑中想吐未吐
只想你和伴侣要好才顽强病好

不聚不散 只等你给另一对手擒获
以为青丝 不会用上余生来量度
但我拖着躯壳 发现沿途寻找的快乐
仍系于你肩膊 或是其实在等我舍割
然后断线风筝会直飞天国

一直不觉 捆绑我的未可扣紧承诺
满头青丝 想到白了仍懒得脱落
被你牵动思觉 最后谁愿缠绕到天国
然后撕裂躯壳 欲断难断在 不甘心去舍割
难道爱本身可爱在于束缚
无奈你我牵过手 没绳索
------------------------------------------------

In [72]:
data_config = {'seq_len':30, 'batch_size':64}
model_config = {'embedding_dim':128, 'hidden_dim':64, 'vocab_size':None}
train_config = {'epochs':10, 'lr':1e-3}
START = '<START>'
STOP = '<STOP>'

class Sequences(Dataset):
    
    def __init__(self, seq_len, raw_data):
        self.seq_len = seq_len
        self.raw_data = raw_data
        self.char2idx = {START:0, STOP:1}
        
        self.sequences = []
        for instance in tqdm_notebook(raw_data):
            for i in set(instance):
                if i not in self.char2idx:
                    self.char2idx[i] = len(self.char2idx)
            
            instance = ['<START>'] + list(instance) + ['<STOP>']
            for i in range(len(instance) - seq_len):
                self.sequences.append(instance[i:i + seq_len])
        
        self.sequences = np.array([[self.char2idx[i] for i in seq] for seq in self.sequences])       
        model_config['vocab_size'] = len(self.char2idx)
        
    def __getitem__(self, idx):
        return self.sequences[idx, :-1], self.sequences[idx, 1:]
    
    def __len__(self):
        return len(self.sequences)
        

In [73]:
dataset = Sequences(seq_len=data_config['seq_len'], raw_data=jay_song)
print(len(dataset))
train_loader = DataLoader(dataset, batch_size=data_config['batch_size'])

35146


In [74]:
print(dataset.char2idx)
print(dataset[0])

{'<START>': 0, '<STOP>': 1, '这': 2, '损': 3, '道': 4, '你': 5, '曲': 6, '格': 7, '再': 8, '泪': 9, '秘': 10, '恋': 11, '亦': 12, '世': 13, '缺': 14, '球': 15, '够': 16, '有': 17, '对': 18, '必': 19, '连': 20, '丢': 21, '念': 22, '使': 23, '密': 24, '前': 25, '共': 26, '胆': 27, '声': 28, '就': 29, '抚': 30, '难': 31, '不': 32, '要': 33, '席': 34, '问': 35, '走': 36, '抵': 37, '出': 38, '还': 39, '词': 40, '吠': 41, '力': 42, '脸': 43, '舒': 44, '伦': 45, '作': 46, '着': 47, '何': 48, '生': 49, '\n': 50, '只': 51, '闭': 52, '别': 53, '个': 54, '见': 55, '地': 56, '福': 57, '便': 58, '孤': 59, '介': 60, '日': 61, ' ': 62, '献': 63, '周': 64, '誓': 65, '今': 66, '离': 67, '可': 68, '面': 69, '眼': 70, '卑': 71, '没': 72, '累': 73, '浊': 74, '身': 75, '快': 76, '留': 77, '自': 78, '试': 79, '给': 80, '物': 81, '的': 82, '算': 83, '毒': 84, '挂': 85, '夕': 86, '也': 87, '失': 88, '下': 89, '起': 90, '礼': 91, '意': 92, '晚': 93, '上': 94, '度': 95, '宁': 96, '是': 97, '尊': 98, '嘶': 99, '想': 100, '现': 101, '资': 102, '废': 103, '无': 104, ':': 105, '哭': 106, '熟': 107, '较': 108, '以': 10

In [75]:
class RNN_LM(nn.Module):
    
    def __init__(self, hidden_dim, vocab_size, embedding_dim, device='cpu'):
        super(RNN_LM, self).__init__()
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.device = device
        
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.rnn = nn.GRU(self.embedding_dim, self.hidden_dim, batch_first=True)
        self.linear = nn.Linear(self.hidden_dim, self.vocab_size)
        
    def hidden(self, batch_size):
        return torch.randn(1, batch_size, self.hidden_dim).to(self.device)
    
    def forward(self, x, hidden):
        x_emb = self.embedding(x)
        x_rnn, hidden = self.rnn(x_emb.unsqueeze(1), hidden)
        x_out = self.linear(x_rnn.squeeze(1))
        return x_out, hidden

model = RNN_LM(hidden_dim=model_config['hidden_dim'],
            vocab_size=model_config['vocab_size'],
            embedding_dim=model_config['embedding_dim']).to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=train_config['lr'],
)

RNN_LM(
  (embedding): Embedding(1853, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=1853, bias=True)
)


In [78]:
# train
model.train()
train_losses = []

for epoch in range(train_config['epochs']):
    progress_bar = tqdm_notebook(train_loader, leave=False)
    losses = []
    total = 0
    for inputs, targets in progress_bar:
        batch_size = inputs.size(0)
        hidden = model.hidden(batch_size)

        model.zero_grad()
        
        loss = 0
        for char_idx in range(inputs.size(1)):
            output, hidden = model(inputs[:, char_idx].to(device), hidden)
            loss += criterion(output, targets[:, char_idx].to(device))

        loss.backward()

        optimizer.step()
        
        avg_loss = loss.item() / inputs.size(1)
        
        progress_bar.set_description(f'Loss: {avg_loss:.3f}')
        
        losses.append(avg_loss)
        total += 1
    
    epoch_loss = sum(losses) / total
    train_losses.append(epoch_loss)
        
    tqdm.write(f'Epoch #{epoch + 1}\tTrain Loss: {epoch_loss:.3f}')
    

Epoch #1	Train Loss: 4.743


Epoch #2	Train Loss: 4.338


Epoch #3	Train Loss: 4.011


Epoch #4	Train Loss: 3.738


Epoch #5	Train Loss: 3.507


Epoch #6	Train Loss: 3.308


Epoch #7	Train Loss: 3.133


Epoch #8	Train Loss: 2.978


Epoch #9	Train Loss: 2.840


Epoch #10	Train Loss: 2.716


In [ ]:
model.eval()
char2idx = dataset.char2idx
idx2char = {v:k for k,v in char2idx.items()}

temperature = 1.0
seed = START
text = ''
with len(text) < 200:
    batch_size = 1
    hidden = model.hidden(batch_size)
    last_char = char2idx[seed]
    while True:
        output, hidden = model(torch.LongTensor([last_char]).to(device), hidden)
        
        distribution = output.squeeze().div(temperature).exp()
        guess = torch.multinomial(distribution, 1).item()
        
        last_char = guess
        cur_char = idx2char[guess]
        if cur_char == STOP:break
        
        text += cur_char
        
print(text)